### Objetivo: modelo de previsão de vendas
##### Autor: Yan Sym

#### Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from simpledbf import Dbf5
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, make_scorer
from hyperopt import fmin, hp, tpe
from math import sqrt
import lightgbm as lgb
import warnings
import random
import gc

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
%matplotlib inline

#### Variáveis globais e parâmetros

In [2]:
random_seed = 42

lista_vars_explicativas = ['uf',
'NUMTRABINF',
'NUMPOBRESX',
'NUMRENDA',
'valor_servicos',
'valor_pib_per_capta',
'feature_04',
'feature_05',
'feature_06',
'feature_07',
'feature_08',
'feature_09',
'feature_10',
'feature_13',
'feature_14',
'feature_17',
'feature_18',
'feature_04_vezes_06']

params = {'subsample': 0.8,
'reg_lambda': 5,
'reg_alpha': 1,
'num_leaves': 12,
'min_child_weight': 0.7,
'min_child_samples': 10,
'colsample_bytree': 0.6,
'learning_rate': 0.01,
'n_estimators': 1300}

#### Métodos

In [3]:
class CatEncoder():
    
    def __init__(self):
        self.dic = {}
        self.rev_dic = {}
        
    def fit(self, vet):
        uniques = vet.unique()
        for a, b in enumerate(uniques):
            self.dic[b] = a
            self.rev_dic[a] = b
        return self
    
    def check(self, vet):
        if type(vet) == list:
            return pd.Series(vet)
        return vet
    
    def transform(self, vet):
        vet = self.check(vet)
        return vet.map(self.dic).replace(np.nan, -1).astype(int)
    
    def inverse_transform(sekf, vet):
        vet = self.check(vet)
        return vet.map(self.rev_dic).replace(np.nan, 'NaN')

# trata codigo do municipio
def trata_municipio(df):
    df['cod_municipio_6_digitos'] = df['cod_municipio'].astype(str)

    lista_municipios_6_digitos = []

    for municipio in df['cod_municipio'].values.tolist():
        municipio = str(municipio)
        if len(municipio) > 6:
            lista_municipios_6_digitos.append(int(municipio[:-1]))
        else:
            lista_municipios_6_digitos.append(municipio)

    df['cod_municipio_6_digitos'] = lista_municipios_6_digitos
    df['cod_municipio_6_digitos'] = df['cod_municipio_6_digitos'].astype(int)
    return df
    
# métricas de erro
def mae_score(true, pred):
    return mean_absolute_error(true, pred)

def mse_score(true, pred):
    return mean_squared_error(true, pred)

def rmse_score(true, pred):
    return (sqrt(mean_squared_error(true, pred)))

#### Leitura de bases

In [4]:
df_lojas = pd.read_csv('lojas_atuais.csv')
print (df_lojas.shape)
df_lojas = trata_municipio(df_lojas)
df_lojas.head()

(3130, 21)


,cod_loja,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,cod_municipio_6_digitos
0,1,4125506005002,4125506,NaN,FEATURE_02_VALUE_04,True,21.76,FEATURE_05_VALUE_04,31,0.168922,0.167659,115.958848,2.349821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,412550
1,2,5300108005004,5300108,NaN,FEATURE_02_VALUE_03,True,22.00,FEATURE_05_VALUE_03,27,0.215967,0.197287,1562.674817,3.670883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,530010
2,3,3550308005052,3550308,NaN,FEATURE_02_VALUE_03,True,45.66,FEATURE_05_VALUE_03,6,0.190102,0.179140,5593.905463,2.815883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,355030
3,4,3145604003002,3145604,NaN,FEATURE_02_VALUE_04,False,30.00,FEATURE_05_VALUE_03,3,0.141988,0.143677,12.824022,7.218132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,314560
4,5,1600303005010,1600303,FEATURE_01_VALUE_10,FEATURE_02_VALUE_02,False,62.49,FEATURE_05_VALUE_03,30,0.107619,0.103083,325.531120,1.972944,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,100.0,88.1,100.0,100.0,88.5,93.8,160030


In [5]:
# base de dados pública do IBGE (relativos a 2015)
df = Dbf5('RENDABR10.dbf')
df_municipios = pd.read_excel('df_municipios.xlsx')
df_municipios = df_municipios.groupby(['MUNCOD']).mean().reset_index()
print (df_municipios.shape)

for column in df_municipios.columns.tolist():
    if column == 'NUMRENDA':
        df_municipios[column] = df_municipios[column].astype(float)
    else:
        df_municipios[column] = df_municipios[column].astype(int)
        
df_lojas = pd.merge(df_lojas, df_municipios, how='left', left_on='cod_municipio_6_digitos', right_on='MUNCOD')
print (df_lojas.shape)

# base de dados pública do ibge, com informações socioeconômicas de cada município
df_pib = pd.read_excel('df_ibge_2015.xlsx')
df_lojas = pd.merge(df_lojas, df_pib, how='left', left_on='cod_municipio', right_on='cod_municipio')
print (df_lojas.shape)
df_lojas.head()

(5565, 14)
(3130, 36)
(3130, 53)


,cod_loja,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,cod_municipio_6_digitos,MUNCOD,ANO,CORRACA,NUMRENDA,DENRENDA,DENCRIREND,NUMPOBRES,NUMPOBRESX,NUMCRIPOB,NUMCRIPOBX,NUMDESOCUP,DENDESOCUP,NUMTRABINF,DENTRABINF,uf,nome,flag_amazonia,flag_semiarido,flag_sp,valor_agropecuaria,valor_industria,valor_servicos,valor_administracao,valor_adicionado,valor_impostos,valor_pib,valor_pib_per_capta,atividade_1,atividade_2,atividade_3,populacao
0,1,4125506005002,4125506,NaN,FEATURE_02_VALUE_04,True,21.76,FEATURE_05_VALUE_04,31,0.168922,0.167659,115.958848,2.349821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,412550,412550.0,2010.0,2.0,3.653353e+07,43865.0,11050.0,6136.0,1731.0,2393.0,618.0,1089.0,23967.0,451.0,4888.0,PR,São José dos Pinhais,0.0,0.0,0.0,360437.265,6.289830e+06,8.617474e+06,1.573847e+06,1.684159e+07,5.739605e+06,2.258119e+07,73427.61,Demais serviços,Indústrias de transformação,Comércio e reparação de veículos automotores e...,297895.0
1,2,5300108005004,5300108,NaN,FEATURE_02_VALUE_03,True,22.00,FEATURE_05_VALUE_03,27,0.215967,0.197287,1562.674817,3.670883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,530010,530010.0,2010.0,2.0,7.060813e+08,423966.0,101062.0,77681.0,28689.0,29735.0,10787.0,18031.0,230356.0,2892.0,43877.0,DF,Brasília,0.0,0.0,0.0,828313.642,8.443861e+06,1.082876e+08,9.800259e+07,2.155623e+08,2.912043e+07,2.446828e+08,80502.47,"Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...,2789761.0
2,3,3550308005052,3550308,NaN,FEATURE_02_VALUE_03,True,45.66,FEATURE_05_VALUE_03,6,0.190102,0.179140,5593.905463,2.815883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,355030,355030.0,2010.0,2.0,2.636974e+09,1862101.0,388246.0,372964.0,177763.0,125227.0,53378.0,70980.0,985472.0,11427.0,172724.0,SP,São Paulo,0.0,0.0,1.0,37648.699,5.911981e+07,4.746460e+08,4.295861e+07,5.767621e+08,1.225262e+08,6.992884e+08,57759.39,Demais serviços,Comércio e reparação de veículos automotores e...,"Administração, defesa, educação e saúde públic...",11967825.0
3,4,3145604003002,3145604,NaN,FEATURE_02_VALUE_04,False,30.00,FEATURE_05_VALUE_03,3,0.141988,0.143677,12.824022,7.218132,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,314560,314560.0,2010.0,3.0,4.566713e+06,7857.0,1649.0,2050.0,551.0,708.0,213.0,194.0,3996.0,36.0,764.0,MG,Oliveira,0.0,0.0,0.0,52492.165,1.164816e+05,4.281522e+05,1.660616e+05,7.631875e+05,8.481704e+04,8.480046e+05,20235.39,Demais serviços,"Administração, defesa, educação e saúde públic...",Comércio e reparação de veículos automotores e...,41562.0
4,5,1600303005010,1600303,FEATURE_01_VALUE_10,FEATURE_02_VALUE_02,False,62.49,FEATURE_05_VALUE_03,30,0.107619,0.103083,325.531120,1.972944,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,100.0,88.1,100.0,100.0,88.5,93.8,160030,160030.0,2010.0,3.0,5.457762e+07,79093.0,24747.0,31536.0,15347.0,13028.0,6728.0,4089.0,36036.0,866.0,10682.0,AP,Macapá,1.0,0.0,0.0,57994.153,6.288244e+05,4.633651e+06,3.911056e+06,9.231525e+06,7.633520e+05,9.994877e+06,21054.88,"Administração, defesa, educação e saúde públic...",Demais serviços,Comércio e reparação de veículos automotores e...,456171.0


In [6]:
df_resultados_parte_1 = pd.read_csv('desafio_1.csv', sep=';')
df_lojas = pd.merge(df_lojas, df_resultados_parte_1, how='left', on='cod_loja')
df_lojas['feature_04_vezes_06'] = df_lojas['feature_04'] * df_lojas['feature_06']
print (df_lojas.shape)

(3130, 55)


In [7]:
df_lojas = df_lojas.loc[~df_lojas['faturamento'].isnull()]
print (df_lojas.shape)

(3123, 55)


In [8]:
# bases com o cenário de expansão
df_expansao = pd.read_csv('cenarios_expansao.csv')

In [9]:
df_expansao.shape

(553, 21)

In [10]:
df_expansao.head()

,cod_loja,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18
0,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,False,50.00,FEATURE_05_VALUE_03,6,0.134654,0.131481,116.194058,2.913498,FEATURE_11_VALUE_01,FEATURE_12_VALUE_01,100.0,93.4,100.0,100.0,71.7,88.8
1,13,2916401003002,2916401,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,False,41.00,FEATURE_05_VALUE_03,4,0.116662,0.114995,26.345742,1.312612,NaN,NaN,84.3,77.5,100.0,100.0,63.5,78.8
2,16,2928703003003,2928703,NaN,FEATURE_02_VALUE_04,False,42.12,FEATURE_05_VALUE_05,2,0.104162,0.087807,35.362788,1.969512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21,2906006003001,2906006,NaN,FEATURE_02_VALUE_04,False,30.00,FEATURE_05_VALUE_03,6,0.132954,0.131330,9.862757,1.383681,NaN,NaN,85.0,76.9,100.0,100.0,100.0,92.9
4,28,3550308005058,3550308,FEATURE_01_VALUE_10,FEATURE_02_VALUE_02,True,44.31,FEATURE_05_VALUE_03,8,0.190102,0.179140,5593.905463,2.815883,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_expansao = trata_municipio(df_expansao)
print (df_expansao.shape)
df_expansao.head()

(553, 22)


,cod_loja,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,cod_municipio_6_digitos
0,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,False,50.00,FEATURE_05_VALUE_03,6,0.134654,0.131481,116.194058,2.913498,FEATURE_11_VALUE_01,FEATURE_12_VALUE_01,100.0,93.4,100.0,100.0,71.7,88.8,293330
1,13,2916401003002,2916401,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,False,41.00,FEATURE_05_VALUE_03,4,0.116662,0.114995,26.345742,1.312612,NaN,NaN,84.3,77.5,100.0,100.0,63.5,78.8,291640
2,16,2928703003003,2928703,NaN,FEATURE_02_VALUE_04,False,42.12,FEATURE_05_VALUE_05,2,0.104162,0.087807,35.362788,1.969512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,292870
3,21,2906006003001,2906006,NaN,FEATURE_02_VALUE_04,False,30.00,FEATURE_05_VALUE_03,6,0.132954,0.131330,9.862757,1.383681,NaN,NaN,85.0,76.9,100.0,100.0,100.0,92.9,290600
4,28,3550308005058,3550308,FEATURE_01_VALUE_10,FEATURE_02_VALUE_02,True,44.31,FEATURE_05_VALUE_03,8,0.190102,0.179140,5593.905463,2.815883,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,NaN,NaN,NaN,NaN,NaN,NaN,355030


In [12]:
# cruza com a primeira base do ibge
df_expansao = pd.merge(df_expansao, df_municipios, how='left', left_on='cod_municipio_6_digitos', right_on='MUNCOD')
print (df_expansao.shape)

# cruza com a segunda base do ibge
df_expansao = pd.merge(df_expansao, df_pib, how='left', left_on='cod_municipio', right_on='cod_municipio')
print (df_expansao.shape)

(553, 36)
(553, 53)


In [13]:
df_expansao.shape

(553, 53)

In [14]:
df_expansao.head()

,cod_loja,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,cod_municipio_6_digitos,MUNCOD,ANO,CORRACA,NUMRENDA,DENRENDA,DENCRIREND,NUMPOBRES,NUMPOBRESX,NUMCRIPOB,NUMCRIPOBX,NUMDESOCUP,DENDESOCUP,NUMTRABINF,DENTRABINF,uf,nome,flag_amazonia,flag_semiarido,flag_sp,valor_agropecuaria,valor_industria,valor_servicos,valor_administracao,valor_adicionado,valor_impostos,valor_pib,valor_pib_per_capta,atividade_1,atividade_2,atividade_3,populacao
0,10,2933307005008,2933307,FEATURE_01_VALUE_08,FEATURE_02_VALUE_02,False,50.00,FEATURE_05_VALUE_03,6,0.134654,0.131481,116.194058,2.913498,FEATURE_11_VALUE_01,FEATURE_12_VALUE_01,100.0,93.4,100.0,100.0,71.7,88.8,293330,293330.0,2010.0,2.0,2.758921e+07,50766.0,12750.0,21562.0,8900.0,7670.0,3592.0,2269.0,24895.0,510.0,5479.0,BA,Vitória da Conquista,0.0,1.0,0.0,93646.819,8.401360e+05,3.696978e+06,1.077987e+06,5.708747e+06,7.739153e+05,6.482663e+06,18589.99,Demais serviços,Comércio e reparação de veículos automotores e...,"Administração, defesa, educação e saúde públic...",343230.0
1,13,2916401003002,2916401,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,False,41.00,FEATURE_05_VALUE_03,4,0.116662,0.114995,26.345742,1.312612,NaN,NaN,84.3,77.5,100.0,100.0,63.5,78.8,291640,291640.0,2010.0,3.0,6.765195e+06,13593.0,3435.0,5316.0,1748.0,1859.0,659.0,500.0,6618.0,111.0,1513.0,BA,Itapetinga,0.0,1.0,0.0,36561.110,2.974788e+05,3.921075e+05,2.533011e+05,9.794486e+05,1.025668e+05,1.082015e+06,13955.55,Demais serviços,Indústrias de transformação,"Administração, defesa, educação e saúde públic...",76184.0
2,16,2928703003003,2928703,NaN,FEATURE_02_VALUE_04,False,42.12,FEATURE_05_VALUE_05,2,0.104162,0.087807,35.362788,1.969512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,292870,292870.0,2010.0,3.0,9.504002e+06,18065.0,4236.0,7620.0,2962.0,2450.0,1089.0,978.0,9337.0,219.0,1904.0,BA,Santo Antônio de Jesus,0.0,0.0,0.0,57998.091,2.643169e+05,1.182924e+06,3.130932e+05,1.818333e+06,2.762788e+05,2.094611e+06,20268.73,Demais serviços,Comércio e reparação de veículos automotores e...,"Administração, defesa, educação e saúde públic...",101548.0
3,21,2906006003001,2906006,NaN,FEATURE_02_VALUE_04,False,30.00,FEATURE_05_VALUE_03,6,0.132954,0.131330,9.862757,1.383681,NaN,NaN,85.0,76.9,100.0,100.0,100.0,92.9,290600,290600.0,2010.0,3.0,3.515516e+06,13270.0,3888.0,9417.0,5859.0,3304.0,2295.0,457.0,5739.0,293.0,1792.0,BA,Campo Formoso,0.0,1.0,0.0,42810.551,5.202211e+05,2.577472e+05,2.198049e+05,1.040584e+06,6.785107e+04,1.108435e+06,15091.42,Indústrias de transformação,"Eletricidade e gás, água, esgoto, atividades d...","Administração, defesa, educação e saúde públic...",72271.0
4,28,3550308005058,3550308,FEATURE_01_VALUE_10,FEATURE_02_VALUE_02,True,44.31,FEATURE_05_VALUE_03,8,0.190102,0.179140,5593.905463,2.815883,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,NaN,NaN,NaN,NaN,NaN,NaN,355030,355030.0,2010.0,2.0,2.636974e+09,1862101.0,388246.0,372964.0,177763.0,125227.0,53378.0,70980.0,985472.0,11427.0,172724.0,SP,São Paulo,0.0,0.0,1.0,37648.699,5.911981e+07,4.746460e+08,4.295861e+07,5.767621e+08,1.225262e+08,6.992884e+08,57759.39,Demais serviços,Comércio e reparação de veículos automotores e...,"Administração, defesa, educação e saúde públic...",11967825.0


#### separa base em treino e teste

In [15]:
df_train = df_lojas.copy()

In [16]:
df_test = df_expansao.copy()

In [17]:
df_test['feature_04_vezes_06'] = df_test['feature_04'] * df_test['feature_06']

In [18]:
df_train.shape

(3123, 55)

In [19]:
df_test.shape

(553, 54)

### Divide dados em variáveis explicativas e target

In [20]:
X_train = df_train[lista_vars_explicativas].copy()
y_train = df_train[['faturamento']].copy()

X_test = df_test[lista_vars_explicativas].copy()

print (f'X_train: {X_train.shape}')
print (f'y_train: {y_train.shape}')
print (f'X_test: {X_test.shape}')

X_train: (3123, 18)
y_train: (3123, 1)
X_test: (553, 18)


In [21]:
cat_cols = [col for col in X_train.dtypes[X_train.dtypes == 'object'].index]
print (cat_cols)

['uf', 'feature_05']


In [22]:
# converte variáveis categóricas
mapeamento = {}
for col in cat_cols:
    ce = CatEncoder()
    print (f'Criando mapeamento para coluna {col}')
    ce.fit(X_train[col].astype(str))
    mapeamento[col] = ce
     
for col in cat_cols:
    ce = mapeamento[col]
    X_train[col] = ce.transform(X_train[col].astype(str))

for col in cat_cols:
    ce = mapeamento[col]
    X_test[col] = ce.transform(X_test[col].astype(str))

Criando mapeamento para coluna uf
Criando mapeamento para coluna feature_05


#### Fit do modelo

In [23]:
clf = lgb.LGBMRegressor(**params)
clf.fit(X_train, y_train)

LGBMRegressor(colsample_bytree=0.6, learning_rate=0.01, min_child_samples=10,
              min_child_weight=0.7, n_estimators=1300, num_leaves=12,
              reg_alpha=1, reg_lambda=5, subsample=0.8)

#### Resultados de treino

In [24]:
y_pred = clf.predict(X_train)
print ('MAE:', mae_score(y_train, y_pred))
print ('MSE:', mse_score(y_train, y_pred))
print ('RMSE:', rmse_score(y_train, y_pred))

MAE: 15553.282992162505
MSE: 509750634.9516085
RMSE: 22577.657871258667


#### Importância das variáveis explicativas

In [25]:
i = 1
for a,b in sorted(list(zip(clf.booster_.feature_importance(), X_train.columns))):
    print (i, a, b, sep=' ')
    i+=1

1 404 feature_13
2 467 feature_17
3 594 feature_07
4 623 feature_05
5 628 feature_09
6 654 feature_18
7 678 feature_14
8 684 NUMTRABINF
9 705 feature_06
10 754 uf
11 773 NUMPOBRESX
12 777 feature_10
13 806 feature_08
14 846 valor_servicos
15 900 NUMRENDA
16 930 valor_pib_per_capta
17 1397 feature_04
18 1680 feature_04_vezes_06


#### Resultados

In [26]:
df_resultados = df_test.copy()
y_pred = clf.predict(X_test)
df_resultados['pred'] = y_pred
df_resultados = df_resultados.reset_index()
df_resultados = df_resultados[['cod_loja', 'pred']].copy()
df_resultados.columns = ['cod_loja', 'faturamento']

In [27]:
df_resultados.head(100)

,cod_loja,faturamento
0,10,174102.033503
1,13,110957.559641
2,16,96665.508499
3,21,74480.015647
4,28,172980.624626
...,...,...
95,646,131112.852717
96,652,204918.099176
97,653,82170.427803
98,678,72412.959658


In [28]:
df_resultados.shape

(553, 2)

In [29]:
#df_resultados['faturamento'] = 0

In [30]:
df_resultados.shape

(553, 2)

#### salva base de submissao do desafio 2

In [31]:
df_resultados.head(10)

,cod_loja,faturamento
0,10,174102.033503
1,13,110957.559641
2,16,96665.508499
3,21,74480.015647
4,28,172980.624626
5,41,225336.486202
6,55,112984.138410
7,64,207136.458205
8,73,190507.480653
9,75,133201.483032


In [32]:
df_resultados.to_csv('desafio_2.csv', sep=';', index=False)